<a href="https://colab.research.google.com/github/danliima/AulasCSS/blob/master/Machine_Learning_Otimiza%C3%A7%C3%A3o_de_modelos_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install graphviz==0.9
!pip install pydot

!apt-get install graphviz

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

dados.head()

In [ ]:
#Situação de "azar" onde as classes estão ordenadas por padrão
dados_azar =  dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=10, return_train_score=False)
media = results["test_score"].mean()
desvio_padrao = results["test_score"].std()
print("Accuracy com cross validate: 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))


In [ ]:
#gerando dados aleatórios de modelo de carro para simulação de agrupamento ao usar nosso estimador
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

In [ ]:
def imprime_resultados(results):
    media = results["test_score"].mean() * 100
    desvio = results["test_score"].std() * 100
    print("Accuracy médio: %.2f" % media)
    print("Intervalo [%.2f, %.2f]" % (media - 2 * desvio, media + 2 * desvio))

In [ ]:
#GroupKFold para analisar como o modelo se comporta com novos modelos
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

In [ ]:
#GroupKFold em um pipeline com StandarScaler e SVC

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

In [ ]:
#GroupKFold para analisar como o modelo se comporta com novos modelos
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

modelo.fit(x_azar, y_azar)
features = x_azar.columns
dot_data = export_graphviz(modelo, out_file=None, filled=True, rounded=True, class_names=["não", "sim"] ,feature_names=features)
grafico = graphviz.Source(dot_data)
grafico

In [ ]:
#testando com a profundidade max de 3
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=3)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

In [ ]:
modelo.fit(x_azar, y_azar)
features = x_azar.columns
dot_data = export_graphviz(modelo, out_file=None, filled=True, rounded=True, class_names=["não", "sim"] ,feature_names=features)
grafico = graphviz.Source(dot_data)
grafico

In [ ]:
#testando com a profundidade max de 10
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=10)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

#Explorando os hiperparâmetros Parâmetros em uma dimensão

In [ ]:
#testando com a profundidade max de 10
def roda_arvore_de_decisao(max_depth):
  SEED = 301
  np.random.seed(SEED)

  cv = GroupKFold(n_splits=10)
  modelo = DecisionTreeClassifier(max_depth=max_depth)
  results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=True)
  train_score = results["train_score"].mean() * 100
  test_score = results["test_score"].mean() * 100
  print("árvore max_depth = %d, treino = %.2f, teste = %.2f" % (max_depth , train_score, test_score))
  tabela = [max_depth, train_score, test_score]
  return tabela

  imprime_resultados(results)

resultados = [roda_arvore_de_decisao(i) for i in range(1, 33)]
resultados = pd.DataFrame(resultados, columns=["max_depth", "train", "test"])
resultados.head()


#Overfit: Ficou "perfeito" para o treino e ruim para o teste

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.lineplot(x="max_depth", y="train", data=resultados)
sns.lineplot(x="max_depth", y="test", data=resultados)
plt.legend(["Treino", "Teste"])


In [ ]:
resultados.sort_values("test", ascending=False).head()

#Explorando os hiperparâmetros Parâmetros em 2 dimensão

In [ ]:
#testando com a profundidade max de 10
def roda_arvore_de_decisao(max_depth, min_samples_leaf):
  SEED = 301
  np.random.seed(SEED)

  cv = GroupKFold(n_splits=10)
  modelo = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
  results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=True)
  train_score = results["train_score"].mean() * 100
  test_score = results["test_score"].mean() * 100
  print("árvore max_depth = %d, min_samples_leaf = %d,treino = %.2f, teste = %.2f" % (max_depth, min_samples_leaf, train_score, test_score))
  tabela = [max_depth, min_samples_leaf,train_score, test_score]
  return tabela

  imprime_resultados(results)


def busca():
  resultados = []
  for max_depth in range(1, 33):
    for min_samples_leaf in [32, 64, 128, 256]:
      tabela = roda_arvore_de_decisao(max_depth, min_samples_leaf)
      resultados.append(tabela)
  resultados = pd.DataFrame(resultados, columns=["max_depth","min_samples_leaf", "train", "test"])
  return resultados

resultados = busca()
resultados.head()

In [ ]:
resultados.sort_values("test", ascending=False).head()

In [ ]:
corr = resultados.corr()
corr

In [ ]:
sns.heatmap(corr, annot=True)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.pairplot(resultados)
plt.show()

In [ ]:
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
def busca():
  resultados = []
  for max_depth in range(1, 33):
    for min_samples_leaf in [128, 192, 256, 512]:
      tabela = roda_arvore_de_decisao(max_depth, min_samples_leaf)
      resultados.append(tabela)
  resultados = pd.DataFrame(resultados, columns=["max_depth","min_samples_leaf", "train", "test"])
  return resultados

resultados = busca()
resultados.head()

In [ ]:
corr = resultados.corr()
corr

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
resultados.sort_values("test", ascending=False).head()

#Explorando 3 dimensões de hiper parâmetros

In [ ]:
#testando com a profundidade max de 10
def roda_arvore_de_decisao(max_depth, min_samples_leaf, min_samples_split):
  SEED = 301
  np.random.seed(SEED)

  cv = GroupKFold(n_splits=10)
  modelo = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split)
  results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=True)
  fit_time =  results['fit_time'].mean()
  score_time = results['score_time'].mean()
  train_score = results["train_score"].mean() * 100
  test_score = results["test_score"].mean() * 100
  tabela = [max_depth, min_samples_leaf, min_samples_split,train_score, test_score, fit_time, score_time]
  return tabela

def busca():
  resultados = []
  for max_depth in range(1, 33):
    for min_samples_leaf in [32, 64, 128, 256]:
      for min_samples_split in [32, 64, 128, 256]:
        tabela = roda_arvore_de_decisao(max_depth, min_samples_leaf,min_samples_leaf)
        resultados.append(tabela)
  resultados = pd.DataFrame(resultados, columns=["max_depth","min_samples_leaf","min_samples_split","train", "test", "fit_time", "score_time"])
  return resultados

resultados = busca()
resultados.head()

In [ ]:
corr = resultados.corr()
corr

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
resultados.sort_values("test", ascending=False).head()

#Explorando espaço de hiper parâmetros com GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth": [3, 5],
    "min_samples_leaf": [32, 64, 128],
    "min_samples_split": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

busca = GridSearchCV(DecisionTreeClassifier(),
                     espaco_de_parametros,
                     cv=GroupKFold(n_splits=10))
busca.fit(x_azar, y_azar, groups=dados.modelo)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
print(busca.best_params_)
print(busca.best_score_ * 100)

In [ ]:
melhor = busca.best_estimator_
melhor

In [ ]:
from sklearn.metrics import accuracy_score

#Evitar essa abordagem, porque ela está sendo otimista

predicoes = melhor.predict(x_azar)
accuracy =  accuracy_score(y_azar, predicoes)
print("Accuracy: %.2f%%" % (accuracy * 100))

# Como ter uma estimativa sem esse vício nos dados que eu já vi?

- Co caso de cross validation com busca de hiper parâmetros, fazemos uma nova validação cruzada. Chama-se nested cross validation.

In [ ]:
from sklearn.model_selection import cross_val_score

# scores = cross_val_score(melhor, x_azar, y_azar, cv=GroupKFold(n_splits=10))

#Infelismente como o pandas não suporta o nested validation com o groupKfold não conseguimos prever o resultado para novos grupos

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth": [3, 5],
    "min_samples_leaf": [32, 64, 128],
    "min_samples_split": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

busca = GridSearchCV(DecisionTreeClassifier(),
                     espaco_de_parametros,
                     cv=KFold(n_splits=5, shuffle=True))
busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(melhor, x_azar, y_azar, cv=KFold(n_splits=5, shuffle=True))
scores

In [ ]:
def imprime_score(scores):
  media = scores.mean() * 100
  desvio = scores.std() * 100
  print("Accuracy médio: %.2f" % media)
  print("Intervalo [%.2f, %.2f]" % (media - 2 * desvio, media + 2 * desvio))

imprime_score(scores)


In [ ]:
melhor = busca.best_estimator_
print(melhor)

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

features = x_azar.columns
dot_data = export_graphviz(melhor, out_file=None, filled=True, rounded=True, class_names=["não", "sim"] ,feature_names=features)
grafico = graphviz.Source(dot_data)
grafico

#Explorando com valores aleatórios: RandonSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth": [3, 5],
    "min_samples_leaf": [32, 64, 128],
    "min_samples_split": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                     espaco_de_parametros,
                      n_iter=16,
                     cv=KFold(n_splits=5, shuffle=True),
                     random_state=SEED)
busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
scores = cross_val_score(melhor, x_azar, y_azar, cv=KFold(n_splits=5, shuffle=True))
imprime_score(scores)

In [ ]:
melhor = busca.best_estimator_
print(melhor)

In [ ]:
features = x_azar.columns
dot_data = export_graphviz(melhor, out_file=None, filled=True, rounded=True, class_names=["não", "sim"] ,feature_names=features)
grafico = graphviz.Source(dot_data)
grafico

# Customizando o espaço de hiper parâmetros

In [ ]:
from scipy.stats import randint

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth": [3, 5, 10, 15, 20, 30, None],
    "min_samples_leaf": randint(32, 128),
    "min_samples_split": randint(32, 128),
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                     espaco_de_parametros,
                      n_iter=16,
                     cv=KFold(n_splits=5, shuffle=True),
                     random_state=SEED)
busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
scores = cross_val_score(melhor, x_azar, y_azar, cv=KFold(n_splits=5, shuffle=True))
imprime_score(scores)
melhor = busca.best_estimator_
print(melhor)

In [ ]:
resultados_ordenados_pela_media =  resultados.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media.iterrows():
  print("%.3f +-(%.3f) %s" % (linha.mean_test_score, linha.std_test_score * 2, linha.params))

#Exploração mais a fundo de forma aleatória

In [ ]:
SEED = 564
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth": [3, 5, 10, 15, 20, 30, None],
    "min_samples_leaf": randint(32, 128),
    "min_samples_split": randint(32, 128),
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                     espaco_de_parametros,
                      n_iter=64,
                     cv=KFold(n_splits=5, shuffle=True),
                     random_state=SEED)
busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
resultados_ordenados_pela_media =  resultados.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media.iterrows():
  print("%.3f +-(%.3f) %s" % (linha.mean_test_score, linha.std_test_score * 2, linha.params))

#Comparando GridSearchCV com RandomizedSearsh (1comparação)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import time

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    "n_estimators": [10, 100],
    "max_depth": [3, 5],
    "min_samples_split": [32, 64, 128],
    "min_samples_leaf": [32, 64, 128],
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]
}

tic= time.time()
busca = GridSearchCV(RandomForestClassifier(),
                     espaco_de_parametros,
                     cv=KFold(n_splits=5, shuffle=True))
busca.fit(x_azar, y_azar)
tac = time.time()
tempo_que_passou = tac - tic
print("Tempo gasto: %.2f segundos" % tempo_que_passou)

resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
resultados_ordenados_pela_media =  resultados.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  print("%.3f +-(%.3f) %s" % (linha.mean_test_score, linha.std_test_score * 2, linha.params))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import time

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    "n_estimators": [10, 100],
    "max_depth": [3, 5],
    "min_samples_split": [32, 64, 128],
    "min_samples_leaf": [32, 64, 128],
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]
}

tic= time.time()
busca = RandomizedSearchCV(RandomForestClassifier(),
                     espaco_de_parametros,
                      n_iter = 20,
                     cv=KFold(n_splits=5, shuffle=True))
busca.fit(x_azar, y_azar)
tac = time.time()
tempo_que_passou = tac - tic
print("Tempo gasto: %.2f segundos" % tempo_que_passou)

resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
resultados_ordenados_pela_media =  resultados.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  print("%.3f +-(%.3f) %s" % (linha.mean_test_score, linha.std_test_score * 2, linha.params))

In [ ]:
tic= time.time()
scores = cross_val_score(busca.best_estimator_, x_azar, y_azar, cv=KFold(n_splits=5, shuffle=True))
tac = time.time()
tempo_que_passou = tac - tic
print("Tempo gasto: %.2f segundos" % tempo_que_passou)

imprime_score(scores)
melhor = busca.best_estimator_
print(melhor)

In [ ]:
SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    "n_estimators": randint(10, 101),
    "max_depth": randint(3, 6),
    "min_samples_split": randint(32, 129),
    "min_samples_leaf": randint(32, 129),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]
}

tic= time.time()
busca = RandomizedSearchCV(RandomForestClassifier(),
                     espaco_de_parametros,
                      n_iter = 80,
                     cv=KFold(n_splits=5, shuffle=True))
busca.fit(x_azar, y_azar)
tac = time.time()
tempo_que_passou = tac - tic
print("Tempo gasto: %.2f segundos" % tempo_que_passou)

resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
resultados_ordenados_pela_media =  resultados.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media[-5:].iterrows():
  print("%.3f +-(%.3f) %s" % (linha.mean_test_score, linha.std_test_score * 2, linha.params))

# Se eu não posso ou não consigo usar o cross validation

In [ ]:
# 0.6 treino  => treino
# 0.2 teste    => dev test
# 0.2 validação  => validação

from sklearn.model_selection import train_test_split

SEED = 301
np.random.seed(SEED)

x_treino_teste, x_validacao, y_treino_teste, y_validacao = train_test_split(x_azar, y_azar, test_size=0.2, shuffle=True, stratify=y_azar)
print(x_treino_teste.shape)
print(x_validacao.shape)
print(y_treino_teste.shape)
print(y_validacao.shape)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

espaco_de_parametros = {
    "n_estimators": randint(10, 101),
    "max_depth": randint(3, 6),
    "min_samples_split": randint(32, 129),
    "min_samples_leaf": randint(32, 129),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]
}

splits = StratifiedShuffleSplit(n_splits=1, test_size=0.25)

tic= time.time()
busca = RandomizedSearchCV(RandomForestClassifier(),
                     espaco_de_parametros,
                      n_iter = 5,
                     cv=splits)
busca.fit(x_treino_teste, y_treino_teste)
tac = time.time()
tempo_que_passou = tac - tic
print("Tempo gasto: %.2f segundos" % tempo_que_passou)

resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

In [ ]:
tic= time.time()
scores = cross_val_score(busca, x_validacao, y_validacao, cv=splits)
tac = time.time()
tempo_que_passou = tac - tic
print("Tempo gasto: %.2f segundos" % tempo_que_passou)
scores